# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'guarana_jesus.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo guarana_jesus.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,rt @giussilvestre: os éguas que tão cancelando...,0
1,@whindersson @lilwhind guaraná jesus... show d...,1
2,bissexuais tomando guarana jesus,0
3,@whindersson @lilwhind guaraná jesus é tipo al...,0
4,rt @maickjhonn: piaui nem tem guaraná jesus is...,0


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,foda que ela é coca-cola e eu guaraná jesus😫,1
1,@niyeongs @tealcookie eu gostei de pernambuco ...,0
2,rt @botbissexual: bissexuais tomando guarana j...,0
3,a bandeira do estado do cara parece mais uma l...,0
4,rt @marilhera: @inesteticas cajuína meets guar...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
train['Classificação'] = train['Classificação'].astype('category')
train['Classificação'].cat.categories = ['No', 'Yes']
train['Classificação'] = pd.Categorical(train['Classificação'], categories = ['No', 'Yes'], ordered = False)
train.head()

,Treinamento,Classificação
0,rt @giussilvestre: os éguas que tão cancelando...,No
1,@whindersson @lilwhind guaraná jesus... show d...,Yes
2,bissexuais tomando guarana jesus,No
3,@whindersson @lilwhind guaraná jesus é tipo al...,No
4,rt @maickjhonn: piaui nem tem guaraná jesus is...,No


In [6]:
import re

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed


for i in range(320):
    train = train.replace(train['Treinamento'][i], cleanup(train['Treinamento'][i]))
    train['Treinamento'] = train['Treinamento'].str.replace('[\n]', '')
    
    if train['Treinamento'][i][:2] == 'rt':
        train = train.replace(train['Treinamento'][i], train['Treinamento'][i][3:])
        
    train = train.replace(train['Treinamento'][i], train['Treinamento'][i].lower())

train.head(10)

,Treinamento,Classificação
0,@giussilvestre os éguas que tão cancelando a t...,No
1,@whindersson @lilwhind guaraná jesus show de b...,Yes
2,bissexuais tomando guarana jesus,No
3,@whindersson @lilwhind guaraná jesus é tipo al...,No
4,@maickjhonn piaui nem tem guaraná jesus isso é...,No
5,a brusinha de frio do guaraná jesus por favor ...,No
6,@badtux_ guaraná jesus é muito injusto vc apel...,Yes
7,@flyingpeachess jogar muito jogo de terror tam...,No
8,@whindersson @lilwhind você não sabe oquanto e...,Yes
9,o moletom guaraná jesus customizado senhorrrr ...,No


In [7]:
all_tweets = []

i = 0
while i < 320:
    all_tweets.append(train['Treinamento'][i].split())
    i+=1
    
train['separados'] = 'Nan'

i = 0
while i < 320:
    train['separados'][i] = all_tweets[i]
    i+=1
    
all_words = []

i = 0 
while i < 320:
    j = 0
    while j < len(all_tweets[i]):
        all_words.append(all_tweets[i][j])
        j+=1
    i+=1
    
train.head()

,Treinamento,Classificação,separados
0,@giussilvestre os éguas que tão cancelando a t...,No,"[@giussilvestre, os, éguas, que, tão, cancelan..."
1,@whindersson @lilwhind guaraná jesus show de b...,Yes,"[@whindersson, @lilwhind, guaraná, jesus, show..."
2,bissexuais tomando guarana jesus,No,"[bissexuais, tomando, guarana, jesus]"
3,@whindersson @lilwhind guaraná jesus é tipo al...,No,"[@whindersson, @lilwhind, guaraná, jesus, é, t..."
4,@maickjhonn piaui nem tem guaraná jesus isso é...,No,"[@maickjhonn, piaui, nem, tem, guaraná, jesus,..."


In [8]:
serie_train = pd.Series(all_words)

tabela_train_relativa = serie_train.value_counts(True)
tabela_train_relativa

jesus                    0.069987
guaraná                  0.063481
de                       0.028264
do                       0.025123
e                        0.021086
                           ...   
vendido                  0.000224
@royairene               0.000224
https//tco/bceba7nilo    0.000224
antes                    0.000224
curto                    0.000224
Length: 1424, dtype: float64

In [12]:
train_R = train.loc[train['Classificação'] == 'Yes', :]
train_Rc = train.loc[train['Classificação'] == 'No', :]

relevant_words = []

for frase in train_R['separados']:
    j = 0
    while j < len(frase):
        relevant_words.append(frase[j])
        j+=1

irrelevant_words = []

for frase in train_Rc['separados']:
    j = 0
    while j < len(frase):
        irrelevant_words.append(frase[j])
        j+=1
        
train_R.head()

,Treinamento,Classificação,separados
1,@whindersson @lilwhind guaraná jesus show de b...,Yes,"[@whindersson, @lilwhind, guaraná, jesus, show..."
6,@badtux_ guaraná jesus é muito injusto vc apel...,Yes,"[@badtux_, guaraná, jesus, é, muito, injusto, ..."
8,@whindersson @lilwhind você não sabe oquanto e...,Yes,"[@whindersson, @lilwhind, você, não, sabe, oqu..."
12,off namoraria comigonome vitóriaaltura 152cor ...,Yes,"[off, namoraria, comigonome, vitóriaaltura, 15..."
16,tive q abrir outra garrafa de guaraná jesus kk...,Yes,"[tive, q, abrir, outra, garrafa, de, guaraná, ..."


In [18]:
serie_train_R = pd.Series(relevant_words)
tabela_train_R_relativa = serie_train_R.value_counts(True)

serie_train_Rc = pd.Series(irrelevant_words)
tabela_train_Rc_relativa = serie_train_Rc.value_counts(True)


0.005931784478497282

In [11]:
P_R = len(serie_train_R) / len(serie_train)
P_Rc = len(serie_train_Rc) / len(serie_train)
print(P_R, P_Rc)

0.4537909376401974 0.5462090623598026


In [20]:
train['fator de prob Relevante'] = P_R
train['fator de prob Irrelevante'] = P_Rc

for indice, linha in train.iterrows():
    for palavra in linha['separados'][indice]:
        linha['fator de prob Relevante'][indice] *= tabela_train_R_relativa[palavra]
        linha['fator de prob Irrelevante'][indice] *= tabela_train_Rc_relativa[palavra]

TypeError: 'float' object is not subscriptable

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**